In [5]:
import random
import numpy as np

In [6]:
class Dice:
    registered = 0

    def __init__(self, probs):
        self.probs = probs
        self._id = Dice.registered
        Dice.registered += 1
    
    def roll(self):
        max_die = len(self.probs)
        min_die = 1
        return min_die + np.random.choice(max_die, 1, p=self.probs)[0]
    
    def __eq__(self, other):
        return self._id == other._id
    
    def __hash__(self):
        return self._id


honest_dice = Dice([1/6 for i in range(6)])
dishonest_dice = Dice([1/10 for i in range(5)] + [1/2])

In [7]:
dice_change_from_p = {
    honest_dice: 0.04,
    dishonest_dice: 0.05
}

dice_change_p_matrix = np.array([ 
        [1-dice_change_from_p[honest_dice], dice_change_from_p[honest_dice]], 
        [dice_change_from_p[dishonest_dice], 1-dice_change_from_p[dishonest_dice]]])

In [10]:
def change_dice(dice):
    if dice == honest_dice:
        return dishonest_dice
    return honest_dice


def roll(n):
    current_dice = random.choice([honest_dice, dishonest_dice])
    rolls = np.zeros(n)
    dice_used = ['' for i in range(n)]
    for i in range(n):
        if random.random() < dice_change_from_p[current_dice]:
            current_dice = change_dice(current_dice)
        rolls[i] = current_dice.roll()
        dice_used[i] = current_dice
    return rolls, dice_used

In [11]:
n = 10000
rolls, dice_used = roll(n)
for die in range(1, 7):
    print(die, np.sum(rolls==die) / n)

1 0.136
2 0.1393
3 0.1407
4 0.131
5 0.1362
6 0.3168


In [42]:
def count_six_dies(rolls):
    return len(list(filter(lambda roll: roll == 6, rolls)))


def heuristic(rolls, interval, constant = 2/3):
    n = len(rolls)
    guesses = ['' for i in range(n)]
    for i in range(n):
        rolls_in_a_row = rolls[max(i - interval, 0) : min(i + interval, n)]
        expected_amount_of_six = constant * interval
        guesses[i] = dishonest_dice if count_six_dies(rolls_in_a_row) > expected_amount_of_six else honest_dice
    return guesses

In [20]:
def get_score(dice_used, guesses):
    correct = 0
    for i in range(len(rolls)):
        if guesses[i] == dice_used[i]:
            correct += 1
    return correct / len(rolls)

In [50]:
for interval in range(6, 12):
    guesses = heuristic(rolls, interval, 2/3)
    score = get_score(dice_used, guesses)
    print(interval, '-', score)

6 - 0.8232
7 - 0.8302
8 - 0.8299
9 - 0.8249
10 - 0.8215
11 - 0.8178


In [44]:
for interval in range(6, 12):
    guesses = heuristic(rolls, interval, 3/4)
    score = get_score(dice_used, guesses)
    print(interval, '-', score)

6 - 0.8232
7 - 0.8162
8 - 0.805
9 - 0.8249
10 - 0.8151
11 - 0.8058


In [16]:
magical_constant = 5.25

In [17]:
def get_roll_prob(dice_id, die):
    dice = honest_dice if dice_id == 0 else dishonest_dice
    return dice.probs[die]


def get_alpha(rolls):
    T = len(rolls)
    alpha = np.zeros((2,T))
    for i in range(T):
        for j in range(2):
            if i==0:
                alpha[j,i] = j
            else:
                for k in range(2):
                    roll = int(rolls[i]) - 1
                    alpha[j,i] += alpha[k,i-1] * dice_change_p_matrix[k,j] * get_roll_prob(k, roll)
                alpha[j,i] *= magical_constant
    return alpha


def get_beta(rolls):
    T = len(rolls)
    beta = np.zeros((2,T))
    for i in range(T-1,-1,-1):
        for j in range(2):
            if i==T-1:
                beta[j,i] = 1
            else:
                for k in range(2):
                    roll = int(rolls[i]) - 1
                    beta[j,i] += beta[k,i+1] * dice_change_p_matrix[j,k] * get_roll_prob(k, roll)
                beta[j,i] *= magical_constant
    return beta


def predict_with_alpha_beta(data):
    alpha = get_alpha(data)
    beta = get_beta(data)
    
    gamma = alpha * beta / np.sum(alpha * beta, axis=0)
    prediction = np.argmax(gamma, axis=0)
    return prediction 

In [18]:
prediction = predict_with_alpha_beta(rolls)
xs = np.array(list(map(lambda dice: dice._id, dice_used)))
print(np.mean(prediction==xs))

0.8382
